In [47]:
'''
Parameters
'''
import requests 
import pandas as pd

'''
Return variables dictionary from JSON url. 
'''
def read_census_json(json_url): 
    
    # Make a GET request to the JSON url 
    response = requests.get(json_url)
    
    # Check if it was successful (status code == 200)
    if response.status_code == 200: 
        
        # This is the initial response, in dictionary form 
        json_dict = response.json()
        
        # There is only one key, so reassign the dictionary variable to that key 
        json_dictionary = json_dict['variables']
    
    else: 
        print("Couldn't read JSON")
    
    return json_dictionary 

'''
Parse the dictionary so that it is intelligible. 
This includes all concepts, and will be filtered by an input group code or concept in the next function.
'''
def return_group_dictionary(json_dictionary): 
    
    # Initialize count 
    count = 0 
    
    # Initialize dictionaries 
    group_dictionary            = {} 
    
    # Iterate through JSON dictionary 
    for code, values in json_dictionary.items(): 
        
        try:
            # Identify group variable 
            group   = values['group']
            
            # Initialize dictionary key for group 
            if group not in group_dictionary.keys() and "," not in group: 
                group_dictionary[group]            = {} 
                group_dictionary[group]['concept'] = []
                group_dictionary[group]['codes']   = []
            else: 
                pass
            
        except: 
            pass 
        
        try: 
            # Identify concept variable 
            concept = values['concept']
            
            # Initialize dictionary key for group 
            group_dictionary[group]['concept'] = concept
            group_dictionary[group]['codes'].append(code)

        except KeyError:
            pass
        
    if group_dictionary != {}: 
        pass
    else: 
        print('No group dictionary')
        
    return group_dictionary

'''
This uses the block and filter words to refine the dictionary created in the previous function and return it as a dataframe of concepts and codes.
'''
def print_group_dictionary(group_dictionary, filter_word, block_words, print_groups, print_codes): 
    
    # Initialize empty lists 
    group_codes = []
    concepts    = []
    
    # For group code, concept/codes dictionary
    for group_code, concept_codes_dict in group_dictionary.items(): 
        go = False 
        
        # Activate by filter word 
        if filter_word in concept_codes_dict['concept'] or filter_word == '': 
            go = True
        
        # Block by block word
        for block_word in block_words: 
            if block_word in concept_codes_dict['concept']:
                go = False
            
        while go == True:
            
            # Print if specified 
            if print_groups == True: 
                print("---------------")
                print(f"Group Code: {group_code}")
                print("---------------")
                
                print("Concept: ")
                print(" > ", concept_codes_dict['concept'])
                print("")

            if print_codes == True: 
                print("Codes: ")
                for code in concept_codes_dict['codes']: 
                    print(" >> ", code)
            
            # Append values to lists for dataframe 
            group_codes.append(group_code)
            concepts.append(concept_codes_dict['concept'])
                
            go = False 
    
    # Construct dataframe 
    acs_dataframe = pd.DataFrame({'Group Codes':group_codes, 'Concepts':concepts})
            
    return acs_dataframe

'''
ACS 2 - MASTER 
'''
def view_acs_codes(json_url, filter_word, blocked_words): 
    
    # Read JSON 
    json_dictionary         = read_census_json(json_url)
    group_dictionary        = return_group_dictionary(json_dictionary)
    acs_dataframe          = print_group_dictionary(group_dictionary, filter_word, blocked_words, print_groups = False, print_codes = False)
    
    return json_dictionary, group_dictionary, acs_dataframe 